In [13]:
import requests
from bs4 import BeautifulSoup
import MetaTrader5 as mt5
import pandas as pd
import pandas_ta as ta
import time
import numpy as np
from datetime import datetime

In [14]:
# coleta de nome do papel
url = 'https://www.fundamentus.com.br/resultado.php'
agent = {'User-Agent':'Mozzila/5.0'}
resposta = requests.get(url, headers=agent)
soup = BeautifulSoup(resposta.text, 'lxml')
tabela = soup.find_all('table')[0]
df = pd.read_html(str(tabela), decimal=',', thousands='.')[0]
df['Papel']

0       POPR4
1       PMET3
2       MNSA3
3       IVTT3
4       CSTB4
        ...  
980    UBBR11
981     UBBR3
982     GLOB4
983     LWSA3
984    BBTG12
Name: Papel, Length: 985, dtype: object

In [15]:
Symbols = df['Papel']
Symbols

0       POPR4
1       PMET3
2       MNSA3
3       IVTT3
4       CSTB4
        ...  
980    UBBR11
981     UBBR3
982     GLOB4
983     LWSA3
984    BBTG12
Name: Papel, Length: 985, dtype: object

In [64]:
ativos = [tick for tick in Symbols]
ativos

['GNDI3',
 'APER3',
 'CASH3',
 'RCSL3',
 'DASA3',
 'RCSL4',
 'MAPT4',
 'NORD3',
 'FRTA3',
 'ELMD3',
 'EMBR3',
 'RRRP3',
 'VLID3',
 'ORVR3',
 'OPCT3',
 'DTCY3',
 'TELB3',
 'HBSA3',
 'ECOR3',
 'TCNO3',
 'HBRE3',
 'BIOM3',
 'TCNO4',
 'WEST3',
 'FRIO3',
 'MEAL3',
 'AMAR3',
 'MBLY3',
 'TELB4',
 'ECPR4',
 'BAHI3',
 'ENJU3',
 'ALPK3',
 'PINE4',
 'TXRX4',
 'CEED3',
 'CTNM3',
 'BKBR3',
 'IRBR3',
 'AHEB3',
 'ESTR4',
 'SGPS3',
 'SHOW3',
 'OIBR4',
 'CRDE3',
 'AZUL4',
 'MMXM3',
 'VIVR3',
 'CTNM4',
 'TCSA3',
 'GOLL4',
 'OIBR3',
 'AVLL3',
 'HOOT4',
 'PLAS3',
 'COGN3',
 'GSHP3',
 'RPMG3',
 'PMAM3',
 'INEP3',
 'JFEN3',
 'INEP4',
 'ATMP3',
 'BBRK3',
 'MTIG4',
 'IGBR3',
 'BDLL3',
 'BDLL4',
 'SNSY5',
 'PDGR3',
 'SLED3',
 'TEKA3',
 'MTIG3',
 'LLIS3',
 'SLED4',
 'OSXB3',
 'TEKA4',
 'HETA4',
 'MATD3',
 'MWET4',
 'MWET3',
 'MGEL4',
 'CEBR6',
 'CEBR5',
 'CEBR3',
 'GPIV33',
 'RSID3',
 'TPIS3',
 'CTKA4',
 'HBTS5',
 'LUPA3',
 'AZEV4',
 'BRAP3',
 'BRKM6',
 'BAZA3',
 'BRAP4',
 'AZEV3',
 'RNEW4',
 'USIM3',
 'MNDL3',

In [17]:
def get_ohlc(ativo, timeframe, n=5):
    ativo = mt5.copy_rates_from_pos(ativo, timeframe, 0, n)
    ativo = pd.DataFrame(ativo)
    ativo['time'] = pd.to_datetime(ativo['time'], unit='s')
    ativo.set_index('time', inplace=True)
    return ativo

In [36]:
# calculo percentil para determinar intervalo de confiança

def percentil(data, up=90, down=10):
    x = data
    x = x.dropna()
    array_x = np.array(x)
    percentil_up = np.percentile(array_x, up)
    percentil_down = np.percentile(array_x, down)
    
    return [percentil_up, percentil_down]

In [88]:
def caroBarato(tick_ativo, up, down, dias):
    # coleta de dados
    if not mt5.initialize():
        print('initialize() falhou')
        mt5.shutdown()

    df = get_ohlc(tick_ativo, mt5.TIMEFRAME_D1, dias)

    # criação médias
    df['M20'] = ta.sma(df['close'], length=20)
    df['M50'] = ta.sma(df['close'], length=50)
    df['M200'] = ta.sma(df['close'], length=200)

    #criação distancia média
    df['Dist20'] = df['close'] - df['M20'] 
    df['Dist50'] = df['close'] - df['M50']
    df['Dist200'] = df['close'] - df['M200']
    
    # filtro
    df = df[df['real_volume'] > 0]

    # calculo percentil das distancia da média 
    # return percentil_up, percentil_down
    per_m20 = percentil(df['Dist20'], up, down)
    per_m50 = percentil(df['Dist50'], up, down)
    per_m200 = percentil(df['Dist200'], up, down)

    # transformação em um array
    
    # variavel temporaria
    valor = 0

    # logica
    # logica valor = +1 +2 +4 or -1 -2 -4 para diferenciar qual média aponta distancia
    if df['Dist20'][-1] > per_m20[0]:
        valor += 1
    
    if df['Dist50'][-1] > per_m50[0]:
        valor += 2
    
    if df['Dist200'][-1] > per_m200[0]:
        valor += 4
    
    if df['Dist20'][-1] < per_m20[1]:
        valor -= 1
    
    if df['Dist50'][-1] < per_m50[1]:
        valor -= 2
    
    if df['Dist200'][-1] < per_m200[1]:
        valor -= 4

    return valor

In [89]:
caroBarato('PETR4', 95, 5, 500)

4

In [90]:
dicionarioCarobarato = {}
for ativo in ativos:
    try:
        dicionarioCarobarato[ativo] = caroBarato(ativo, 95, 5, 500)
        print(f'COLETA DE {ativo}')
    except:
        print(f'{ativo} SEM COLETA')
        dicionarioCarobarato[ativo] = 'NAN'
dicionarioCarobarato = pd.DataFrame.from_dict(dicionarioCarobarato, orient='index').reset_index()

COLETA DE GNDI3
COLETA DE APER3
COLETA DE CASH3
COLETA DE RCSL3
COLETA DE DASA3
COLETA DE RCSL4
COLETA DE MAPT4
COLETA DE NORD3
COLETA DE FRTA3
COLETA DE ELMD3
COLETA DE EMBR3
COLETA DE RRRP3
COLETA DE VLID3
COLETA DE ORVR3
COLETA DE OPCT3
COLETA DE DTCY3
COLETA DE TELB3
COLETA DE HBSA3
COLETA DE ECOR3
COLETA DE TCNO3
COLETA DE HBRE3
COLETA DE BIOM3
COLETA DE TCNO4
COLETA DE WEST3
COLETA DE FRIO3
COLETA DE MEAL3
COLETA DE AMAR3
COLETA DE MBLY3
COLETA DE TELB4
COLETA DE ECPR4
COLETA DE BAHI3
COLETA DE ENJU3
COLETA DE ALPK3
COLETA DE PINE4
COLETA DE TXRX4
COLETA DE CEED3
COLETA DE CTNM3
COLETA DE BKBR3
COLETA DE IRBR3
COLETA DE AHEB3
COLETA DE ESTR4
COLETA DE SGPS3
COLETA DE SHOW3
COLETA DE OIBR4
COLETA DE CRDE3
COLETA DE AZUL4
COLETA DE MMXM3
COLETA DE VIVR3
COLETA DE CTNM4
COLETA DE TCSA3
COLETA DE GOLL4
COLETA DE OIBR3
COLETA DE AVLL3
COLETA DE HOOT4
COLETA DE PLAS3
COLETA DE COGN3
COLETA DE GSHP3
COLETA DE RPMG3
COLETA DE PMAM3
COLETA DE INEP3
COLETA DE JFEN3
COLETA DE INEP4
COLETA D

In [91]:
dicionarioCarobarato.columns = ["Ticker", "Valor"]
dicionarioCarobarato

,Ticker,Valor
0,GNDI3,0
1,APER3,0
2,CASH3,0
3,RCSL3,0
4,DASA3,0
...,...,...
415,BIDI11,-4
416,AMER3,0
417,VIIA3,0
418,SEQL3,0


In [92]:
ativos_caros = dicionarioCarobarato[dicionarioCarobarato['Valor'] > 4]
ativos_caros.sort_values(by='Valor')

,Ticker,Valor
351,INTB3,5
29,ECPR4,6
132,DMMO3,6
12,VLID3,7
27,MBLY3,7
118,CPLE3,7
125,CPLE6,7


In [93]:
ativos_baratos = dicionarioCarobarato[dicionarioCarobarato['Valor'] < -4]
ativos_baratos.sort_values(by='Valor')

,Ticker,Valor
6,MAPT4,-7
299,WLMM4,-7
241,CGAS5,-7
204,BMEB3,-7
391,BRFS3,-7
168,CSRN5,-6
34,TXRX4,-6
24,FRIO3,-6
383,ALPA4,-6
7,NORD3,-6


In [ ]:
# colocar filtro de liquidez
# colocar coleta yahoo finace